In [62]:
from py2neo import Node, Graph, Relationship
import numpy as np
import pandas as pd

# Connect to your local neo4j database, I excluded authentication requirements for simplicity
graph = Graph("bolt://localhost:7687")

In [63]:
dummy_nodes = [Node("dummy_node", batch_id="batch_1", sample_id="sample_1", step_id = 27),
               Node("dummy_node", batch_id="batch_1", sample_id="sample_2", step_id = 27),
               Node("dummy_node", batch_id="batch_2", sample_id="sample_1", step_id = 27)]

In [64]:
for dummy in dummy_nodes:
    dummy['default_id'] = dummy.identity
    graph.create(dummy)

In [65]:
dummy_batch1 = pd.DataFrame(np.random.rand(10, 2))
dummy_batch2 = pd.DataFrame(np.random.rand(10,1))
dummy_batch1.rename(columns = {0:'sample_1', 1:'sample_2'}, inplace = True)
dummy_batch2.rename(columns = {0:'sample_1'}, inplace = True)

In [79]:
# function to pass in csv and append appropriate columns to nodes
# csv will be converted to a dataframe beforehand
def add_color_data(b_id, df, file_path: None):
    samples= df.columns
    for sample in samples:
        data = list(df.loc[:, sample])
        new_node = Node('dummy_data', action = 'char', batch_id = b_id, sample_id = sample, char_name = 'colormetrics', \
                        char_num_measurements = len(data), char_duration=len(data)*2, char_colormetrics = data, step_id = 32)
        # find all nodes with a sample_id equal to that in the dataframe
        sample_nodes = graph.run("MATCH (n:dummy_node {sample_id: $sample, batch_id: $batch, step_id: 27}) RETURN n", sample = sample, batch = b_id)
        graph.create(new_node)
        # create relatinships between the node and data node
        for node in sample_nodes:
            existing_node = node['n']
            relationship = Relationship(existing_node, 'NEXT', new_node)
            graph.create(relationship)


In [81]:
add_color_data('batch_1', dummy_batch1, None)
add_color_data('batch_2', dummy_batch2, None)